# Test the model on a video or live feed

### Load necessary libraries

In [1]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import model_from_json

c:\Users\Het\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Het\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Het\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Some pre-processing

In [2]:
def get_key(val):
    for key, value in pred_dict.items():
         if val == value:
             return key
 
    return "key doesn't exist"

pred_dict = {'defect_1':0,'defect_2':1,'normal':2}

### Load the json model file

In [3]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


### Run the video and apply the model

In [13]:
# 'http://192.168.0.100:8080/video' for remote else test.mp4
string = 'test.mp4'
cap = cv2.VideoCapture(string)
if (cap.isOpened()== False):
  print("Error opening video stream or file")

while(cap.isOpened()):
  ret, frame = cap.read()

  if ret == True:
    frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)
    
    if circles is not None:
      circles = np.round(circles[0, :]).astype("int")
      for (x, y, r) in circles:
        cv2.circle(frame, (x, y), r, (0, 255, 0), 4)
      
      copy_frame = frame[y-r:y+r,x-r:x+r]
      img = cv2.resize(copy_frame,(168,168))

      X =  image.img_to_array(img)
      X = np.expand_dims(X,axis=0)
      images = np.vstack([X])
      val = np.argmax(loaded_model.predict(images),axis=-1)
      string = get_key(val)
      # Display the resulting frame
      font = cv2.FONT_HERSHEY_DUPLEX
      try:
        cv2.putText(frame, string, (x-r, y-r), font, 1.0, (255, 255, 255), 1)
        cv2.imshow('Test video',frame)        
      except:
        pass
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
 
  else:
    break
 
cap.release()
 
cv2.destroyAllWindows()

1/1 [==============================] - 0s 23ms/step
